# Withdraw Unit Test
Performing verification of the withdraw mechanism by comparing the event data to the results of cadCAD model.

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')

Context(prec=6, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])
Context(prec=6, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])
Context(prec=6, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])
stakeDelegateds: 605 events
stakeDelegatedLockeds: 375 events
stakeDelegatedWithdrawns: 326 events
allocationCloseds: 47 events
allocationCollecteds: 16 events
stakeDepositeds: 25 events
rewardsAssigneds: 47 events
delegationParametersUpdateds: 6 events
TOTAL NUMBER OF EVENTS: 1529
You should set SIMULATION_TIME_STEPS in config.py to a minimum of 1529 to capture all events.



Configure environment

In [2]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [3]:
withdraw_events_dict = withdraw_events

print("EXPECTED TRUTH--Tokens withdrawn via withdraw events:")
for timestep, withdraw_event in withdraw_events_dict.items():
    print(f"{timestep}, {withdraw_event[0]['delegator']}, {withdraw_event[0]['tokens']}")



EXPECTED TRUTH--Tokens withdrawn via withdraw events:
644, 0xf5376e847efa1ea889bfcb03706f414dade0e82c, 12488.567623993900000000
645, 0x1853bf6a892abd949aed052bb634ab96afaf31dd, 14169.714085664846000000
646, 0xd776a7306ee6a060cebb46b46d305e88fd39ba84, 251819.644650000000000000
647, 0x81b4687d4193916aa6cf65360bf96b1e4279278a, 6505.310000000000000000
650, 0x57b4a5f470c85d4281f7baf5162b59243a3491df, 65257.075000000000000000
653, 0xe4369be6df50aa406db8212ae00cf81917c848ea, 63951.707761229540017576
654, 0xfe7c442da0297313444b919ba17ca666d631d121, 18167.407271115301580346
663, 0x13c0dabeb938c21524c59cdc40bcb6fdb3618754, 132706.115100000000000000
664, 0x60696f9d6ab5d99560bd04a98c718731735edd5c, 10185.289624385739620000
665, 0x4e43a0cd79a9324645167e70aa6cdbdfa91a43e9, 650.029186268782600178
669, 0x1eda5b5fc4b6fce3dc442381a46088850f206eca, 19395.188250346788624364
673, 0x625114d3fc10a15a5465d3137aba4771464aae3d, 1297.479999999999999999
674, 0x85194ffba51d0a3d9b7abcae802868f1e03f021c, 99.62934999

In [4]:
print("MODELED RESULTS--Tokens locked in undelegation.")
for timestep, withdraw_event in withdraw_events_dict.items():
    event = withdraw_event[0]
    new_tokens_withdrawn = df.iloc[timestep-1].indexers[event['indexer']].holdings
    old_tokens_withdrawn = df.iloc[timestep-2].indexers[event['indexer']].holdings
    tokens_withdrawn = new_tokens_withdrawn - old_tokens_withdrawn
    print(f"{timestep}, {withdraw_event[0]['delegator']}, {tokens_withdrawn=}")

MODELED RESULTS--Tokens locked in undelegation.


AttributeError: 'Series' object has no attribute 'delegators'

In [ ]:
print("UNITTEST RESULTS")
cntExact = 0
cntReallyClose = 0
cntClose = 0
cnt = 0
cntWrong = 0
for timestep, withdraw_event in withdraw_events_dict.items():
    event = withdraw_event[0]
    new_tokens_withdrawn = df.iloc[timestep-1].indexers[event['indexer']].delegators[event['delegator']].holdings
    old_tokens_withdrawn = df.iloc[timestep-2].indexers[event['indexer']].delegators[event['delegator']].holdings
    tokens_withdrawn = new_tokens_withdrawn - old_tokens_withdrawn
    print(f"Event: Timestep={timestep}, Delegator ID={event['delegator']}, Tokens Withdrawn={event['tokens']}")
    print(f"Model: Timestep={timestep}, Delegator ID={event['delegator']}, Tokens Withdrawn={tokens_withdrawn}")
    # print("Withdraw Amount Equal?", event[0]['tokens'] == tokens_withdrawn)
    # print()
    try:
        ratio =  abs(event['tokens'] / tokens_withdrawn)
    except:
        ratio = np.inf
    exact = ratio == 1.0
    close = 0.99 <= ratio <= 1.01
    reallyClose = 0.99999 <= ratio <= 1.00001

    if exact:            
        cntExact += 1
    elif reallyClose:
        cntReallyClose += 1
    elif close:
        cntClose += 1
    else:
        cntWrong += 1
    cnt += 1
    print("Tokens Equal?", exact)
    print("Tokens Within 0.001%?", reallyClose)
    print("Tokens Within 1%?", close)
    print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   
    print()
print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   



# Conclusion
The number of tokens withdrawn, measured as (holdings_after_event) - (holdings_before_event), is 0 in the first two events, possibly due to the locked_until threshold not being met.  In the next three, we are close, but diverging again.